In [1]:
from config import *
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def get_list_of_floats_from_string(string):
    string = string.replace('[', '').replace(']', '').split(',')
    float_list = [float(i) for i in string]
    return float_list

def get_list_of_strings_from_string(string_list, num_of_stalls):
    string_list = string_list.replace('[', '').replace(']', '').replace("'", "").split(',')
    for i in range(num_of_stalls - len(string_list)):
        string_list.append('') # add empty strings to the list, so that the length of the list is equal to the number of stalls	
    return string_list

def convert_pd_series_to_np_array(pd_series):
    test = pd_series
    test = np.array([get_list_of_floats_from_string(i) for i in test])
    return test

import filepaths

In [2]:
result_parent_directory =  result_parent_directory
path_Sim =  path_Sim
prediction_directory =  prediction_directory
path_DataBase =  path_DataBase

path = result_parent_directory + os.sep + 'step6_MPC_simulation_own_charging_events'

result_directory = path + os.sep + 'simulation_results'
os.makedirs(result_directory, exist_ok=True)
figure_directory = path + os.sep + 'figures'
os.makedirs(figure_directory, exist_ok=True)

sizing_results_stats_and_selected_stations_path = os.path.join(result_parent_directory, 'step4_btms_sizing_sensitivity', 'analysis', 'step4_used_charging_depots_for_control_with_stats_a_5.0.csv')
a_value_selected = 5.0
sizing_results_parent_directory = os.path.join(result_parent_directory, 'step4_btms_sizing_sensitivity', 'sizing_results')

# find out in which folder the results for the selected a value are stored
for folder in os.listdir(sizing_results_parent_directory):
    _path = os.path.join(sizing_results_parent_directory, folder)
    _path = os.listdir(_path)[0]
    _path = os.path.join(sizing_results_parent_directory, folder, _path)
    df = pd.read_csv(_path, index_col=0)
    if df['param: btms size, a,b_sys,b_cap,b_loan,c'].iloc[1] *365/12 == a_value_selected:
        sizing_results_directory = os.path.join(sizing_results_parent_directory, folder)
        break

path_infrastructure = os.path.join(result_parent_directory, 'step2_k_means_clustering', 'infrastructure_removed_zeros.csv')

In [3]:
df_selected_stations_and_stats = pd.read_csv(sizing_results_stats_and_selected_stations_path, index_col=0)
df_selected_stations_and_stats.head()

,btms_size,cost_a,cost_b_sys,cost_b_cap,cost_b_loan,cost_b,cost_c,cost_total,share_cost_a,share_cost_b_sys,...,share_cost_b_loan,share_cost_b,share_cost_c,E_Charge,c_rate,cycles_day,btms_ratio,load_factor,btms_peak_to_grid_ratio,grid_peak_to_charge_peak_ratio
1008,84.285089,8.781543,23.633997,12.186058,4.917977,35.820055,9.448051,54.049649,0.162472,0.437265,...,0.090990,0.662725,0.174803,552.625000,6.484883,5.311781,0.810141,0.492781,10.231527,0.089035
1013,75.758786,13.870309,9.323477,12.319028,4.971641,21.642506,9.541380,45.054194,0.307858,0.206939,...,0.110348,0.480366,0.211776,658.416667,2.846169,5.975156,0.687514,0.364650,2.555441,0.281259
1033,102.514243,3.898475,24.918451,8.037244,3.243623,32.955695,6.234312,43.088482,0.090476,0.578309,...,0.075278,0.764838,0.144686,328.583333,5.621504,2.880149,0.898574,0.669036,24.299674,0.039526
1035,141.328501,5.638278,24.460809,10.058026,4.059159,34.518835,7.798730,47.955843,0.117572,0.510069,...,0.084644,0.719805,0.162623,415.638889,4.002735,2.614599,0.889034,0.584267,16.492952,0.057166
104,47.916710,49.315027,12.971969,2.315360,0.934419,15.287329,1.792749,66.395105,0.742751,0.195375,...,0.014074,0.230248,0.027001,1057.666667,6.260869,1.775664,0.080445,0.149077,1.000002,0.500000


In [4]:
df_infrastructure = pd.read_csv(path_infrastructure, index_col=0)
df_infrastructure.head()

,parkingType,pricingModel,chargingPointType,numStalls,feeInCents,reservedFor,parkingZoneId,locationX,locationY
taz,,,,,,,,,
1008,Workplace,FlatFee,DepotXFC(300.0|DC),6,0,ridehail(GlobalRHM),depot-1008-34,557763.343469,4.200385e+06
1013,Workplace,FlatFee,DepotXFC(300.0|DC),2,0,ridehail(GlobalRHM),depot-1013-128,586536.564655,4.239191e+06
1033,Workplace,FlatFee,DepotXFC(300.0|DC),5,0,ridehail(GlobalRHM),depot-1033-54,576279.733891,4.192507e+06
1034,Workplace,FlatFee,DepotXFC(300.0|DC),2,0,ridehail(GlobalRHM),depot-1034-141,576153.278079,4.166168e+06
1035,Workplace,FlatFee,DepotXFC(300.0|DC),2,0,ridehail(GlobalRHM),depot-1035-94,540607.862798,4.215708e+06


In [5]:
type(df_selected_stations_and_stats.index[0])

numpy.int64

plot power flow at selected charging stations

In [6]:
P_BTMS_color ='#4C72B0'
P_Charge_color ='#55A868'
P_Grid_color ='#C44E52'
P_Charge_Desired_color ='#8172B2'
t_lag_color = '#CCB974'
e_lag_color = '#64B5CD'

In [54]:
stations_to_plot_results = ['972'] #df_selected_stations_and_stats.index.values
#stations_to_plot_results = [str(i) for i in df_selected_stations_and_stats.index.values]

for_paper = True

for station in stations_to_plot_results:
    # load charging station state results
    df_station_state = pd.read_csv(os.path.join(result_directory, station, 'ChargingStationState.csv'), index_col=0)
    time_vector = df_station_state.index / 3600
    num_of_stalls = df_infrastructure.loc[int(station)]['numStalls']
    # start plotting

    # use seaborn style
    plt.style.use('seaborn')
    # box around legend
    plt.rcParams['legend.frameon'] = True
    pltSettings()
    plt.rcParams['font.size'] = 7
    plt.rcParams['mathtext.fontset'] = 'cm'
    # set line width
    plt.rcParams['lines.linewidth'] = 1.0
    if for_paper:
        figsize = (plotSizeX2col, 1.5*plotSizeY2col)
    else:
        figsize = (10,10)
    fig, ax = plt.subplots(3 + num_of_stalls ,1, figsize=figsize, sharex=True, gridspec_kw={'height_ratios': [3,1,1]+ [1 for i in range(num_of_stalls)]})
    pltSettings()
    # first ax: total power flow
    # second ax: E_BTMS of BTMS
    # following ax: power and E_vehicle of each stall
    # last ax: time_lag_sum and energy_lag_sum of all stalls

    # plot total power flow
    ax[0].plot(time_vector, df_station_state['BtmsPower'], label=r'$P_\Theta \mathrm{[kW]}$', color=P_BTMS_color)
    ax[0].plot(time_vector, df_station_state['TotalChargingPowerDesire'], label=r'$P_{Charge,Desired} \mathrm{[kW]}$', color=P_Charge_Desired_color)
    ax[0].plot(time_vector, df_station_state['TotalChargingPower'], label=r'$P_{Charge} \mathrm{[kW]}$', color=P_Charge_color)
    ax[0].plot(time_vector, df_station_state['GridPower'], label=r'$P_{Grid} \mathrm{[kW]}$', color=P_Grid_color)

    #ax[0].set_ylabel('Power [kW]')
    ax[0].set_title('Charging Station TAZ ' + station, fontsize=7)
    ax[0].legend(bbox_to_anchor=(1.08, 1), loc='upper left', borderaxespad=0., fontsize = 7)
    #ax[0].legend(loc='upper right', frameon=True, framealpha=1, edgecolor='black', facecolor='white')

    # plot E_BTMS
    _btms_size = df_selected_stations_and_stats.loc[int(station)]['btms_size']
    ax[1].axhline(y=_btms_size, linestyle='--', color=P_BTMS_color)
    ax[1].text(0.5, 0.05* _btms_size, 'BTMS size: ' + str(round(_btms_size, 2)) + ' kWh', fontsize=7)
    ax[1].axhline(y=0, linestyle='--', color=P_BTMS_color)
    ax[1].plot(time_vector, df_station_state['BtmsEnergy'], label=r'$E_\Theta \mathrm{[kWh]}$')
    ax[1].set_ylim([0, 1.1* _btms_size])
    #ax[1].set_ylabel('Energy [kWh]')
    ax[1].legend(bbox_to_anchor=(1.08, 1), loc='upper left', borderaxespad=0., fontsize = 7)

    # plot power and E_vehicle of each stall
    _bays_charging_power = convert_pd_series_to_np_array(df_station_state['BaysChargingPower'])
    _bays_charging_power_desired = convert_pd_series_to_np_array(df_station_state['BaysChargingDesire'])
    # convert nan to o in _bays_charging_power_desired
    _bays_charging_power_desired[np.isnan(_bays_charging_power_desired)] = 0

    for i in range(num_of_stalls):
        ax[i+2].plot(time_vector, _bays_charging_power[:,i], label=r'$P_{Charge} \mathrm{[kW]}$', color=P_Charge_color)
        ax[i+2].plot(time_vector, _bays_charging_power_desired[:,i], label=r'$P_{Charge,Desired} \mathrm{[kW]}$', color=P_Charge_Desired_color)
        # add text with vehicle id add top of each charging peak in plot, rotate by 90 degree
        # convert to list of vehicle ids
        _vehicle_ids = np.array([get_list_of_strings_from_string(x, num_of_stalls) for x in df_station_state['BaysVehicleIds']])
        #_unique_vehicle_ids = np.unique(_vehicle_ids[:,i])
        # get vehicle id only if vehicle in bay changed
        _unique_vehicle_ids = []
        for j in range(len(_vehicle_ids[:,i])):
            if len(_unique_vehicle_ids) == 0 and _vehicle_ids[j,i] != '':
                _unique_vehicle_ids.append(_vehicle_ids[j,i])
            else:
                if _vehicle_ids[j,i] != _vehicle_ids[j-1,i] and not _vehicle_ids[j,i] == '':
                    _unique_vehicle_ids.append(_vehicle_ids[j,i])
        if len(_unique_vehicle_ids) > 0:
            # for each unique vehicle id, find the time when it is charging
            for _vehicle_id in _unique_vehicle_ids:
                _vehicle_id_charging_time = np.where(_vehicle_ids == _vehicle_id)[0][0]
                _vehicle_id_charging_power = _bays_charging_power[_vehicle_id_charging_time, i]
                # add text at the peak of charging power
                _vehicle_name = _vehicle_id.split('-')[1]
                _vehicle_name = _vehicle_name.split('@')[0]
                y = _vehicle_id_charging_power
                y = 1.05 * _bays_charging_power[:,i].max()
                ax[i+2].text(time_vector[_vehicle_id_charging_time], y, _vehicle_name, rotation=30, fontsize=5, verticalalignment='bottom', horizontalalignment='left')
        #ax[i+2].set_ylabel('Power [kW]')
        ax[i+2].legend(bbox_to_anchor=(1.08, 1), loc='upper left', borderaxespad=0., fontsize = 7)
        ax[i+2].set_ylim([0, 650])

    # plot time_lag_sum and energy_lag_sum of all stalls
    # plo time lag on left y axis
    ax[-1].plot(time_vector, df_station_state['TimeLagSum'], label=r'$\sum_ t_{lag,i}(t) \mathrm{[s]}$', color=t_lag_color)
    # plot energy lag on right y axis
    ax_right = ax[-1].twinx()
    ax_right.plot(time_vector, df_station_state['EnergyLagSum'], label=r'$\sum_ E_{lag,i}(t) \mathrm{[kWh]}$', color=e_lag_color)
    # plot one legend for both y axis
    handles, labels = ax[-1].get_legend_handles_labels()
    handles2, labels2 = ax_right.get_legend_handles_labels()
    handles.extend(handles2)
    labels.extend(labels2)  
    ax[-1].legend(handles, labels, bbox_to_anchor=(1.08, 1), loc='upper left', borderaxespad=0., fontsize = 7)
    #ax[-1].set_ylabel('time lag [s]')
    #ax_right.set_ylabel('energy lag [kWh]')
    ax[-1].set_ylim([0, max([1.1* df_station_state['TimeLagSum'].max(), 600])])
    ax_right.set_ylim([min([1.1* df_station_state['EnergyLagSum'].min(), -0.1*_btms_size]), max([1.1* df_station_state['EnergyLagSum'].max(), 0.3*_btms_size])])

    #set ticks fontsize to 7
    for i in range(len(ax)):
        ax[i].tick_params(axis='y', labelsize=7)
    ax[-1].tick_params(axis='x', labelsize=7)
    ax_right.tick_params(axis='y', labelsize=7)

    

    ax[-1].set_xlabel('Time [h]', fontsize=7)
    ax[-1].set_xlim([0,24])
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.15)
    # save figure
    fig.savefig(figure_directory + os.sep + 'powerflow_charging_station_' + station + '.png', dpi=300, bbox_inches='tight')
    if for_paper:
        fig.savefig(figure_directory + os.sep + 'a_step6_powerflow_charging_station_' + station + '.pdf', dpi=300, bbox_inches='tight')
    plt.close(fig)